In [1]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [2]:
unzip_data("nlp_getting_started.zip")

In [3]:
# Visualiizng the dataset
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [5]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target: {target}","(real disaster)" if target >0 else "(not real disaster)")
    print(f"Text :\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text :
Violent Storm Causes Damage Flooding in Watertown - http://t.co/3ZASZ6wxjJ

---

Target: 1 (real disaster)
Text :
Why did I come to work today.. Literally wanna collapse of exhaustion

---

Target: 0 (not real disaster)
Text :
@emmerdale I'll be devastated if it's cain...such a great character ??

---

Target: 1 (real disaster)
Text :
@2for1Lapdances For the people who died in Human Experiments by Unit 731 of Japanese military http://t.co/vVPLFQv58P http://t.co/yOMPxJpPTV

---

Target: 0 (not real disaster)
Text :
#bigbrother #ch4 The X-37b's big brother revealed: Boeing bags $6.6m contract to design ... - Daily Mail http://t.co/0migwcmtJe

---



In [9]:
#split data into training and validation data
from sklearn.model_selection import train_test_split

In [10]:
train_sen ,val_sen ,train_lab ,val_lab  = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                           train_df_shuffled["target"].to_numpy(),
                                                           test_size=0.1, random_state=42)

In [12]:
len(train_sen)

6851

In [13]:
train_sen[:10],train_lab[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [14]:
#converting text to numbers
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [15]:
text_vectorizer = TextVectorization(max_tokens=10000, 
                                    standardize="lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams=None,output_mode = "int", output_sequence_length = None, pad_to_max_tokens=True)

In [16]:
# Find the average number of tokens (words) in the training 
round(sum([len(i.split()) for i in train_sen])/(len(train_sen)))

15

In [17]:
max_vocab_length = 10000
max_length=15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)

In [18]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sen)

In [19]:
#create a sample sentence and tokenie it
sample = "There's a flood in my street and crying babies!"
text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 264,    3,  232,    4,   13,  698,    7, 1693, 1552,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [20]:
random_sen = random.choice(train_sen)
print(f"Original text:  {random_sen}\n\nVetorized version: {text_vectorizer([random_sen])}")

Original text:  950. If a landslide tumbles down todayI'm on your side

And if a twister sweeps it all away-

YOU'RE ON YOUR OWN BITCH!*runs into distance*

Vetorized version: [[   1   47    3  374 7313  134 7516   11   33  668    7   47    3  358
  7871]]


In [21]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-6:]

In [22]:
top_5_words, bottom_5_words, len(words_in_vocab)

(['', '[UNK]', 'the', 'a', 'in'],
 ['pageshi', 'pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'],
 10000)

In [23]:
#Embedding
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set the input shape
                             output_dim=128, # set the size of the embedding vector
                             embeddings_initializer="uniform", # default, initialize embedding vectors randomly
                             input_length=max_length # how long is each input
                             )

embedding

In [28]:
#get a sandom sentence
random_sentences = random.choice(train_sen)
print(f"OG : {random_sentences}\n\n Embedded version {embedding(text_vectorizer([random_sentences]))}")

OG : Vanessa's game has officially blown up. LADIES AND GENTLEMEN...the real show is about to begin. #BB17

 Embedded version [[[ 0.02114853 -0.01716919  0.02854658 ... -0.02847395 -0.00321012
    0.01920824]
  [ 0.0480498  -0.01387738  0.03236241 ... -0.03914816  0.0086089
    0.03506294]
  [ 0.01261474 -0.03703045 -0.02447871 ... -0.03156354  0.01787206
   -0.03412212]
  ...
  [ 0.03636995  0.02491469 -0.03919793 ...  0.03557464 -0.00642558
   -0.01553101]
  [ 0.01500777 -0.03212347 -0.00874083 ...  0.00442946  0.00128595
   -0.02040151]
  [ 0.04707149  0.03702844 -0.03769959 ... -0.00366423 -0.00658206
   -0.02770127]]]


In [29]:
sample_embed = embedding(text_vectorizer([random_sentences]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02114853, -0.01716919,  0.02854658, ..., -0.02847395,
         -0.00321012,  0.01920824],
        [ 0.0480498 , -0.01387738,  0.03236241, ..., -0.03914816,
          0.0086089 ,  0.03506294],
        [ 0.01261474, -0.03703045, -0.02447871, ..., -0.03156354,
          0.01787206, -0.03412212],
        ...,
        [ 0.03636995,  0.02491469, -0.03919793, ...,  0.03557464,
         -0.00642558, -0.01553101],
        [ 0.01500777, -0.03212347, -0.00874083, ...,  0.00442946,
          0.00128595, -0.02040151],
        [ 0.04707149,  0.03702844, -0.03769959, ..., -0.00366423,
         -0.00658206, -0.02770127]]], dtype=float32)>

In [30]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.02114853, -0.01716919,  0.02854658, -0.02548515,  0.02353642,
        0.01796193,  0.03615484, -0.04391557, -0.02428354,  0.02686692,
        0.04612448,  0.02146241,  0.03267963,  0.03867053,  0.03792075,
        0.01608707, -0.0020406 , -0.0479168 , -0.0051725 , -0.04579569,
       -0.01714941,  0.0051432 , -0.00495577, -0.01212855, -0.03420444,
       -0.03071332, -0.01338337,  0.01596732, -0.02651415, -0.01987622,
        0.00541102, -0.00190752,  0.02519445,  0.03437883,  0.01931479,
       -0.02650642,  0.00724127,  0.04098669, -0.01283176,  0.00601544,
       -0.00425624, -0.00079735, -0.01419212,  0.00849004, -0.00331774,
        0.04298745, -0.00362196, -0.00447987, -0.00722032, -0.03485271,
        0.02625649,  0.00483822, -0.03224083, -0.00594584,  0.02697195,
       -0.04998155, -0.04683535, -0.04412025,  0.03694171, -0.04075827,
        0.01918197, -0.02517774,  0.00776281,  0.04077   , -0.01101736,
        0.041679

# MODELLING 

In [31]:
#Model 0 - baseline model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sen, train_lab )

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [33]:
baseline_score = model_0.score(val_sen,  val_lab)
baseline_score*100

79.26509186351706

In [35]:
baseline_preds = model_0.predict(val_sen)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [37]:
train_lab[:20]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
      dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_resuts(y_true,y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) *100
    model_precision, model_recall, model_fscore , _ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
    
    model_results = {"accuracy": model_accuracy,
                    "precision" : model_precision,
                    "recall": model_recall,
                     "f1" : model_fscore}
    return model_results

In [39]:
baseline_results = calculate_resuts(val_lab,baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [40]:
#MODEL 1 - Dense Model

#create a tensorboard callback
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [41]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs,outputs, name="model_1_dense")

In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [43]:
model_1.compile(loss="binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [44]:
model_1.fit(x=train_sen,y=train_lab,epochs=5,
           validation_data=(val_sen,val_lab),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_1_Dense")])

Saving TensorBoard log files to: model_logs/model_1_Dense/20230920-133635
Epoch 1/5
215/215 [==============================] - 7s 25ms/step - loss: 0.6368 - accuracy: 0.6452 - val_loss: 0.5733 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4722 - accuracy: 0.8264 - val_loss: 0.4768 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3444 - accuracy: 0.8737 - val_loss: 0.4525 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.2607 - accuracy: 0.9088 - val_loss: 0.4538 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.2005 - accuracy: 0.9320 - val_loss: 0.4613 - val_accuracy: 0.8005


In [45]:
model_1.evaluate(val_sen,val_lab)

24/24 [==============================] - 0s 1ms/step - loss: 0.4613 - accuracy: 0.8005


[0.46131521463394165, 0.8005249500274658]

In [46]:
model_1_pred_probs = model_1.predict(val_sen)
model_1_pred_probs.shape, model_1_pred_probs[0]

24/24 [==============================] - 0s 1ms/step


((762, 1), array([0.4601638], dtype=float32))

In [47]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [48]:
model_1_results = calculate_resuts(val_lab,model_1_preds)
model_1_results

{'accuracy': 80.0524934383202,
 'precision': 0.8035781807215083,
 'recall': 0.800524934383202,
 'f1': 0.798485125042071}

In [49]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [50]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [51]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [53]:
embed_weights = model_1.get_layer("embedding").get_weights()

In [61]:
embed_weights[0].shape

(10000, 128)

In [69]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
    #print(index)
    if index == 0:
        continue  # skip 0, it's padding.
    vec = embed_weights[0][index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

In [66]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [68]:
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception:
    pass

In [70]:
#RNN - Recurrent Neural Network

In [74]:
#MODEL 2 LSTM - Long Short Term Memory
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
print(x.shape)
x = embedding(x)
print(x.shape)
x = layers.LSTM(units=64,return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15)
(None, 15, 128)
(None, 15, 64)
(None, 64)


In [75]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                      

In [78]:
model_2.compile(loss="binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [79]:
model_2_history = model_2.fit(train_sen,train_lab,epochs=5,
                             validation_data=(val_sen,val_lab),callbacks = [create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230920-143131
Epoch 1/5
215/215 [==============================] - 12s 40ms/step - loss: 0.3737 - accuracy: 0.8326 - val_loss: 0.4738 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 9s 44ms/step - loss: 0.2476 - accuracy: 0.9026 - val_loss: 0.5340 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1780 - accuracy: 0.9355 - val_loss: 0.5444 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 10s 48ms/step - loss: 0.1378 - accuracy: 0.9545 - val_loss: 0.6352 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1112 - accuracy: 0.9600 - val_loss: 0.8312 - val_accuracy: 0.7677


In [80]:
#make predictions 
model_2_pred_probs = model_2.predict(val_sen)
model_2_pred_probs[:10]*100

24/24 [==============================] - 1s 4ms/step


array([[ 0.8077557 ],
       [95.14631   ],
       [99.79685   ],
       [ 0.45730025],
       [ 0.54192203],
       [99.81753   ],
       [53.001385  ],
       [99.957085  ],
       [99.94677   ],
       [ 4.5288687 ]], dtype=float32)

In [82]:
val_lab[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [87]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_results = calculate_resuts(val_lab,model_2_preds)

In [90]:
model_2_results

{'accuracy': 76.77165354330708,
 'precision': 0.7823170011167755,
 'recall': 0.7677165354330708,
 'f1': 0.7608603371570797}

In [111]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.LSTM(units=128,return_sequences=True)(x)
#x = layers.LSTM(units=64,return_sequences=True)(x)
#x = layers.LSTM(64)(x)
#x = layers.Dense(64, activation="tanh")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2_1 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [114]:
model_2_1.compile(loss="binary_crossentropy",
               optimizer = tf.keras.optimizers.Adam(),
               metrics = ["accuracy"])

In [115]:
model_2_1_history = model_2.fit(train_sen,train_lab,epochs=5,
                             validation_data=(val_sen,val_lab),callbacks = [create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230920-144541
Epoch 1/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0245 - accuracy: 0.9863 - val_loss: 2.0054 - val_accuracy: 0.7467
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0243 - accuracy: 0.9861 - val_loss: 1.9984 - val_accuracy: 0.7336
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0238 - accuracy: 0.9867 - val_loss: 2.1265 - val_accuracy: 0.7454
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0239 - accuracy: 0.9869 - val_loss: 2.1647 - val_accuracy: 0.7454
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0237 - accuracy: 0.9860 - val_loss: 2.2086 - val_accuracy: 0.7428


In [117]:
#MODEL 3 GRU - Gated Recurrent Unit

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64,return_sequences=True)(x)
#x = layers.LSTM(42,return_sequences=True)(x)
#x = layers.GRU(99)(x)
#x = layers.Dense(64,activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [118]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_2 (GRU)                 (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d (  (None, 64)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_19 (Dense)            (None, 1)                 

In [119]:
model_3.compile(loss="binary_crossentropy",
               optimizer = "Adam",
               metrics=["accuracy"])

In [120]:
model_3_history = model_3.fit(train_sen,train_lab,epochs=5, validation_data = (val_sen,val_lab))

Epoch 1/5
215/215 [==============================] - 11s 40ms/step - loss: 0.1944 - accuracy: 0.9277 - val_loss: 0.7633 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0766 - accuracy: 0.9734 - val_loss: 1.0645 - val_accuracy: 0.7533
Epoch 3/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0528 - accuracy: 0.9775 - val_loss: 1.2272 - val_accuracy: 0.7336
Epoch 4/5
215/215 [==============================] - 9s 44ms/step - loss: 0.0434 - accuracy: 0.9806 - val_loss: 1.2067 - val_accuracy: 0.7362
Epoch 5/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0390 - accuracy: 0.9823 - val_loss: 1.4833 - val_accuracy: 0.7520


In [121]:
model_3_pred_prob = model_3.predict(val_sen)
model_3_pred_prob[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.6292954e-04],
       [6.0934973e-01],
       [9.9887830e-01],
       [3.7152380e-02],
       [2.3947605e-04],
       [9.9999529e-01],
       [9.9817419e-01],
       [9.9999416e-01],
       [9.9999291e-01],
       [7.2866991e-02]], dtype=float32)

In [125]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_prob))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [127]:
model_3_results = calculate_resuts(val_lab, model_3_pred)

In [129]:
model_3_results

{'accuracy': 75.19685039370079,
 'precision': 0.7575330533748617,
 'recall': 0.7519685039370079,
 'f1': 0.747484854432977}

In [133]:
#MODEL4 BiDirectional
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs,name="model_4_bidirectional")

In [134]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                                                 
 dense_22 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1354625 (5.17 MB)
Trainable par

In [136]:
model_4.compile(loss="binary_crossentropy",
               optimizer = "Adam",
               metrics=["accuracy"])

In [138]:
model_4_history = model_4.fit(train_sen,train_lab,epochs=6, validation_data=(val_sen,val_lab))

Epoch 1/6
215/215 [==============================] - 13s 44ms/step - loss: 0.1335 - accuracy: 0.9502 - val_loss: 1.0766 - val_accuracy: 0.7651
Epoch 2/6
215/215 [==============================] - 10s 45ms/step - loss: 0.0462 - accuracy: 0.9784 - val_loss: 1.0101 - val_accuracy: 0.7454
Epoch 3/6
215/215 [==============================] - 10s 45ms/step - loss: 0.0371 - accuracy: 0.9813 - val_loss: 1.3225 - val_accuracy: 0.7493
Epoch 4/6
215/215 [==============================] - 10s 46ms/step - loss: 0.0323 - accuracy: 0.9823 - val_loss: 1.4053 - val_accuracy: 0.7520
Epoch 5/6
215/215 [==============================] - 10s 46ms/step - loss: 0.0307 - accuracy: 0.9819 - val_loss: 1.3060 - val_accuracy: 0.7415
Epoch 6/6
215/215 [==============================] - 10s 47ms/step - loss: 0.0376 - accuracy: 0.9815 - val_loss: 1.4293 - val_accuracy: 0.7507


In [139]:
model_4_pred_probs = model_4.predict(val_sen)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[3.5633839e-04],
       [9.5224088e-01],
       [9.9994653e-01],
       [1.1921198e-01],
       [1.1720209e-04],
       [9.9981481e-01],
       [9.9765605e-01],
       [9.9999213e-01],
       [9.9998957e-01],
       [7.9106176e-01]], dtype=float32)

In [143]:
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [144]:
model_4_results = calculate_resuts(val_lab,model_4_pred)

In [145]:
model_4_results

{'accuracy': 75.06561679790026,
 'precision': 0.7511704709318223,
 'recall': 0.7506561679790026,
 'f1': 0.7487994714665389}

In [146]:
#MODEL 5 Conv1D

embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32,kernel_size=5,activation="relu",padding="valid")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [147]:
embedding_test, conv_1d_output

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.06314449, -0.12392423,  0.04029237, ..., -0.05040809,
          -0.08832544, -0.14402828],
         [-0.05903536, -0.15985551,  0.03025048, ..., -0.06139763,
          -0.03200936, -0.08880804],
         [-0.11328594, -0.11914556, -0.06083946, ..., -0.00295749,
          -0.08070169, -0.19928318],
         ...,
         [-0.04981161, -0.1416941 ,  0.07250482, ..., -0.03215808,
          -0.08027259, -0.13103412],
         [-0.04981161, -0.1416941 ,  0.07250482, ..., -0.03215808,
          -0.08027259, -0.13103412],
         [-0.04981161, -0.1416941 ,  0.07250482, ..., -0.03215808,
          -0.08027259, -0.13103412]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.        , 0.25098237, 0.        ,
          0.        , 0.        , 0.02643723, 0.12282584, 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.0522891 ,
          0.        , 0.